In [1]:
import os
import cv2
import torch
import glob
import numpy as np
from torchvision import transforms
from xml.etree import ElementTree as ET

In [2]:
# Ścieżli datasetu
DATASET_PATH = "LLVIP"
IMG_V_PATH = os.path.join(DATASET_PATH, "visible", "train")
IMG_IR_PATH = os.path.join(DATASET_PATH, "infrared", "train")
ANNOTATIONS_PATH = os.path.join(DATASET_PATH, "Annotations")

#### Transformacje dla YOLO
wymagany input i normalizacja do zakresu [-1, 1]

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((416, 416)),
    transforms.Normalize([0.5], [0.5])
])

#### Funckje do konwersji danych


In [4]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import os

def parse_voc_annotation(xml_path):
    """Parsuje plik XML z adnotacjami w formacie VOC"""
    tree = ET.parse(xml_path)
    root = tree.getroot()

    labels = []
    for obj in root.findall("object"):
        label = obj.find("name").text
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        labels.append((label, xmin, ymin, xmax, ymax))

    return labels
    
def load_image_pairs(img_name, dataset_path):
    """ Wczytuje parę obrazów RGB i IR, skaluje i łączy w jeden tensor. """
    img_rgb = Image.open(os.path.join(dataset_path, "visible/train", img_name)).convert("RGB")
    img_ir = Image.open(os.path.join(dataset_path, "infrared/train", img_name)).convert("L")  # Skala szarości

    transform = transforms.Compose([
        transforms.Resize((416, 416)),  # Przeskalowanie do wymiarów YOLO
        transforms.ToTensor()
    ])

    img_rgb = transform(img_rgb)  # (3, 416, 416)
    img_ir = transform(img_ir)  # (1, 416, 416)

    img_fused = torch.cat((img_rgb, img_ir), dim=0)  # (4, 416, 416)
    return img_fused

In [59]:
# Test: Wczytanie jednej próbki
sample_name = "010001.jpg"  # Nazwa przykładowego obrazu
img_fused = load_image_pairs(sample_name, DATASET_PATH)
annotations = parse_voc_annotation(os.path.join(ANNOTATIONS_PATH, "010001.xml"))

print(f"Rozmiar obrazu po fuzji: {img_fused.shape}")  # Powinno być (4, 416, 416)
print(f"Adnotacje: {annotations}")

Rozmiar obrazu po fuzji: torch.Size([4, 416, 416])
Adnotacje: [('person', 287, 428, 351, 662), ('person', 351, 391, 424, 642), ('person', 466, 367, 550, 614), ('person', 700, 354, 761, 585), ('person', 704, 517, 806, 794), ('person', 1124, 22, 1196, 245)]


#### Łączenie obrazów w obraz 4-ro kanałowy

In [ ]:
# import os
# import cv2
# import torch
# import numpy as np
# from torchvision import transforms
# from tqdm import tqdm  # Pasek postępu

# # Ścieżki
# DATASET_PATH = "LLVIP"
# SAVE_PATH = "datasets/LLVIP_fused"

# # Transformacja obrazu do formatu PyTorch
# transform = transforms.Compose([
#     transforms.ToTensor(),  # Zamiana na tensor (0,1)
#     transforms.Resize((416, 416))  # Resize do 416x416
# ])

# # Funkcja do przetwarzania i zapisu obrazów
# def process_and_save_images(split):
#     input_v_path = os.path.join(DATASET_PATH, "visible", split)
#     input_ir_path = os.path.join(DATASET_PATH, "infrared", split)
#     output_path = os.path.join(SAVE_PATH, split)

#     os.makedirs(output_path, exist_ok=True)

#     image_list = sorted(os.listdir(input_v_path))

#     for img_name in tqdm(image_list, desc=f"Przetwarzanie {split}"):
#         # Wczytanie obrazów RGB i IR
#         img_v_path = os.path.join(input_v_path, img_name)
#         img_ir_path = os.path.join(input_ir_path, img_name)

#         img_v = cv2.imread(img_v_path)  # RGB (3 kanały)
#         img_ir = cv2.imread(img_ir_path, cv2.IMREAD_GRAYSCALE)  # IR (1 kanał)

#         if img_v is None or img_ir is None:
#             print(f"POMINIĘTO {img_name} (brak pliku)")
#             continue

#         # Konwersja do tensora
#         img_v = transform(img_v)
#         img_ir = transform(img_ir).squeeze(0)  # Usunięcie zbędnego wymiaru

#         # Łączenie jako (RGB + IR) → 4 kanały
#         img_fused = torch.cat((img_v, img_ir.unsqueeze(0)), dim=0)


#         # Konwersja do NumPy
#         img_fused_np = (img_fused.numpy() * 255).astype(np.uint8)  # (4, 416, 416)

#         # Zamiana kolejności osi do OpenCV: (C, H, W) → (H, W, C)
#         img_fused_np = np.transpose(img_fused_np, (1, 2, 0))

#         # **Zapis do formatu obsługującego 4 kanały (PNG)**
#         save_img_path = os.path.join(output_path, img_name.replace(".jpg", ".tif"))
#         cv2.imwrite(save_img_path, img_fused_np)

#         # **Sprawdzenie po zapisie**
#         img_check = cv2.imread(save_img_path, cv2.IMREAD_UNCHANGED)

#     print(f"Zapisano obrazy do {output_path}")

# # Przetwarzanie zbiorów train i test
# process_and_save_images("train")
# process_and_save_images("test")


Przetwarzanie train: 100%|██████████| 12025/12025 [06:06<00:00, 32.78it/s]


✅ Zapisano obrazy do datasets/LLVIP_fused\train


Przetwarzanie test: 100%|██████████| 3463/3463 [01:44<00:00, 33.18it/s]

✅ Zapisano obrazy do datasets/LLVIP_fused\test


In [ ]:
# Kod do konwersji adnotacji z formatu VOC do YOLO

# import os
# import xml.etree.ElementTree as ET

# # Ścieżki do katalogów
# ANNOTATIONS_PATH = "LLVIP/Annotations"
# DATASET_PATH = "datasets/LLVIP_fused/train"  # Tylko dla zbioru treningowego

# # Pobranie listy klas
# CLASSES = ["person"]  # Możesz dodać więcej klas

# # Funkcja konwertująca XML do formatu YOLO
# def convert_voc_to_yolo(xml_file, output_txt):
#     tree = ET.parse(xml_file)
#     root = tree.getroot()

#     img_width = int(root.find("size/width").text)
#     img_height = int(root.find("size/height").text)

#     with open(output_txt, "w") as f:
#         for obj in root.findall("object"):
#             class_name = obj.find("name").text
#             if class_name not in CLASSES:
#                 continue  # Pomijamy nieznane klasy

#             class_id = CLASSES.index(class_name)

#             bbox = obj.find("bndbox")
#             xmin = int(bbox.find("xmin").text)
#             ymin = int(bbox.find("ymin").text)
#             xmax = int(bbox.find("xmax").text)
#             ymax = int(bbox.find("ymax").text)

#             # YOLO format (normalizacja do [0,1])
#             x_center = (xmin + xmax) / (2 * img_width)
#             y_center = (ymin + ymax) / (2 * img_height)
#             bbox_width = (xmax - xmin) / img_width
#             bbox_height = (ymax - ymin) / img_height

#             f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

# # Przetwarzanie zbioru treningowego
# for img_file in os.listdir(DATASET_PATH):
#     if img_file.endswith(".tif") or img_file.endswith(".jpg"):  # Sprawdzamy tylko obrazy
#         base_name = os.path.splitext(img_file)[0]  # Usuwamy rozszerzenie
#         xml_path = os.path.join(ANNOTATIONS_PATH, f"{base_name}.xml")
#         txt_path = os.path.join(DATASET_PATH, f"{base_name}.txt")
#         print(f"🔍 Sprawdzam: {img_file} → {xml_path}")  # DODANE LOGOWANIE

#         if os.path.exists(xml_path):
#             convert_voc_to_yolo(xml_path, txt_path)
#             print(f"✅ Przetworzono: {img_file} → {base_name}.txt")
#         else:
#             print(f"⚠️ Brak anotacji dla {img_file} (oczekiwano {xml_path})")

# print("🎯 Konwersja zakończona!")


In [5]:
import os

DATASET_PATH = "datasets/LLVIP_fused"

for split in ["train", "test"]:
    missing_labels = []
    img_folder = os.path.join(DATASET_PATH, split)

    for img_file in os.listdir(img_folder):
        if img_file.endswith(".png"):
            txt_file = img_file.replace(".png", ".txt")
            if not os.path.exists(os.path.join(img_folder, txt_file)):
                missing_labels.append(img_file)

    if missing_labels:
        print(f"Brakuje etykiet dla {len(missing_labels)} obrazów w {split}:")
        print(missing_labels[:10])  # Pokażemy pierwsze 10 brakujących
    else:
        print(f"Wszystkie obrazy w {split} mają etykiety!")



Wszystkie obrazy w train mają etykiety!
Wszystkie obrazy w test mają etykiety!


### Dostosowanie modelu pod obraz 4-ro kanałowy
1. Zmiana 1 warstwy
2. wymuszenie obslugi 4 kanałow

In [6]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# Wczytanie modelu
model3 = YOLO("yolov3-sppu.pt").to("cuda")  # Przenosimy cały model na GPU

# Pobranie pierwszej warstwy konwolucyjnej
conv1 = model3.model.model[0].conv  

# Nowa warstwa
new_conv1 = nn.Conv2d(
    in_channels=4, 
    out_channels=conv1.out_channels, 
    kernel_size=conv1.kernel_size, 
    stride=conv1.stride, 
    padding=conv1.padding, 
    bias=conv1.bias is not None
)

# Kopiowanie wag RGB, IR losowo
with torch.no_grad():
    new_conv1.weight[:, :3] = conv1.weight
    new_conv1.weight[:, 3] = torch.randn_like(conv1.weight[:, 0]) * 0.01  # IR losowe wagi

model3.model.model[0].conv = new_conv1.to("cuda")

model3.overrides['imgsz'] = 416  # Rozmiar wejścia
model3.overrides['augment'] = True  # Włącz augmentację
model3.save("yolov3-4ch.pt")
# Sprawdzenie poprawności
print("Nowa pierwsza warstwa:", model3.model.model[0].conv)


Nowa pierwsza warstwa: Conv2d(4, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


In [7]:
# Funkcja do sprawdzania wymiarów wag
def check_conv_weights():
    w = model3.model.model[0].conv.weight
    print(f"First layer weights shape: {w.shape}")  # Powinno być (out_channels, 4, kernel_size, kernel_size)

# Wczytanie modelu po modyfikacji, żeby uniknąć resetowania przy każdym `train()`
model3 = YOLO("yolov3-4ch.pt").to("cuda")

# Trenowanie modelu YOLOv3 na 4-kanałowych obrazach
model3.train(data="llvip.yaml", epochs=10, imgsz=416, batch=8, device="cuda")

# Sprawdzenie po zakończeniu treningu
check_conv_weights()


Ultralytics 8.3.94  Python-3.12.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolov3-4ch.pt, data=llvip.yaml, epochs=10, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning D:\Projects\multimodal\datasets\LLVIP_fused\train.cache... 12025 images, 2 backgrounds, 0 corrupt: 100%|██████████| 12025/12025 [00:00<?, ?it/s]
val: Scanning D:\Projects\multimodal\datasets\LLVIP_fused\test.cache... 3463 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3463/3463 [00:00<?, ?it/s]


Plotting labels to runs\detect\train7\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 85 weight(decay=0.0), 92 weight(decay=0.0005), 91 bias(decay=0.0)
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to runs\detect\train7
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/10      4.72G       2.02      1.575      1.745          4        416: 100%|██████████| 1504/1504 [06:04<00:00,  4.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 217/217 [00:43<00:00,  4.99it/s]


                   all       3463       8302      0.578      0.549       0.56      0.269

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      6.37G      1.491     0.7278      1.383          3        416: 100%|██████████| 1504/1504 [05:31<00:00,  4.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 217/217 [00:43<00:00,  4.98it/s]

                   all       3463       8302       0.87       0.84       0.89      0.523



10 epochs completed in 1.048 hours.
Optimizer stripped from runs\detect\train7\weights\last.pt, 209.8MB
Optimizer stripped from runs\detect\train7\weights\best.pt, 209.8MB

Validating runs\detect\train7\weights\best.pt...
Ultralytics 8.3.94  Python-3.12.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
YOLOv3-spp summary (fused): 100 layers, 104,714,099 parameters, 0 gradients, 283.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 217/217 [00:42<00:00,  5.07it/s]


                   all       3463       8302      0.871      0.839       0.89      0.524
Speed: 0.1ms preprocess, 7.4ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train7
First layer weights shape: torch.Size([32, 3, 3, 3])


In [76]:
# Trenowanie modelu YOLOv3 na 4-kanałowych obrazach


model3.train(data="llvip.yaml", epochs=25, imgsz=416, batch=8, device="cuda")


SyntaxError: '[31m[1mchannels[0m' is not a valid YOLO argument. 

    Arguments received: ['yolo', '--f=c:\\Users\\Radosz\\AppData\\Roaming\\jupyter\\runtime\\kernel-v3104cfdbea569844c11f4ab8cce28fc55a88dea40.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'obb', 'segment', 'classify', 'detect', 'pose'})
                MODE (required) is one of frozenset({'predict', 'val', 'benchmark', 'export', 'train', 'track'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=

In [ ]:
import torch
import torch.nn as nn
from ultralytics import YOLO

# Załaduj model
model11 = YOLO("yolo11n.pt").to("cuda")

# Pobierz pierwszą warstwę konwolucyjną
conv1 = model.model.model[0].conv

# Nowa warstwa 4-kanałowa
new_conv1 = nn.Conv2d(
    in_channels=4, 
    out_channels=conv1.out_channels, 
    kernel_size=conv1.kernel_size, 
    stride=conv1.stride, 
    padding=conv1.padding, 
    bias=conv1.bias is not None
)

# Kopiowanie wag RGB, a kanał IR inicjalizowany losowo
with torch.no_grad():
    new_conv1.weight[:, :3] = conv1.weight  # Skopiuj RGB
    new_conv1.weight[:, 3] = torch.randn_like(conv1.weight[:, 0]) * 0.01  # Losowe wartości dla IR

# Podstawienie nowej warstwy
model11.model.model[0].conv = new_conv1.to("cuda")

# Sprawdzenie
print(model.model.model[0].conv)


Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)


In [72]:
import cv2
import numpy as np
from ultralytics.data import YOLODataset

class CustomYOLODataset(YOLODataset):
    def load_image(self, i):
        path = self.im_files[i]
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)  # Wczytanie 4-kanałowego obrazu
        if img is None:
            raise ValueError(f"Nie można wczytać obrazu: {path}")
        return img, cv2.imread(path).shape[:2]  # Zachowanie oryginalnych wymiarów

from ultralytics.engine.trainer import BaseTrainer

class CustomTrainer(BaseTrainer):
    def get_dataloader(self, dataset_path, batch_size, img_size, augment):
        return CustomYOLODataset(dataset_path, img_size, batch_size, augment)


In [ ]:
model11.train(trainer=CustomTrainer, data="llvip.yaml", epochs=50, imgsz=416, batch=8, device="cuda", verbose=True)


Ultralytics 8.3.94  Python-3.12.9 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8191MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=llvip.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

NotImplementedError: This task trainer doesn't support loading cfg files